# model

> Fill in a module description here

In [ ]:
#| default_exp model

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
import torch.nn.functional as F
from torchtyping import TensorType

In [ ]:
#| export
class Head(nn.Module):
    def __init__(
        self, head_size: int,
        n_embd: int, context_length: int,
        dropout: int=0.1
    ):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(context_length, context_length)))
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: TensorType["batch_size", "seq_len", "n_channels"]):
        B, T, C = x.shape

        k = self.key(x)
        q = self.query(x)

        # compute attention scores
        # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = q @ k.transpose(-2, -1) / (C ** 0.5)

        # masked future scores
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = self.dropout(wei)

        wei = F.softmax(wei, dim=-1)

        v = self.value(x) # (B, T, C)
        out = wei @ v # (B, T, C)
        return out

In [ ]:
#| export
class MultiHeadAttention(nn.Module):
    def __init__(
        self,
        n_heads: int, head_size: int,
        n_embd: int, context_length: int,
        dropout: int=0.1
    ) -> None:
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, context_length) for _ in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return out

In [ ]:
#| export
class FeedForward(nn.Module):
    def __init__(self, n_embd: int, dropout: int=0.1) -> None:
        super().__init__()
        self.net = nn.Sequential(
            # d_ff in the paper is 2048
            nn.Linear(n_embd, n_embd * 4),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
#| export
class Block(nn.Module):
    def __init__(
        self, n_head: int,
        n_embd: int, context_length: int
    ) -> None:
        super().__init__()
        
        head_size = n_embd // n_head

        self.attn = MultiHeadAttention(n_head, head_size, n_embd, context_length=context_length)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
#| export
class GPT2(nn.Module):
    def __init__(
        self,
        n_head: int, n_layer: int,
        vocab_size: int, n_embd: int, context_length: int
    ):
        """_summary_

        Args:
            vocab_size (int): The number of vocabulary
            n_embd (int): The dimension embedding for each token
            context_length (int): The number of tokens to look back
        """
        super().__init__()
        
        head_size = n_embd // n_head

        self.context_length = context_length

        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(context_length, n_embd)
        self.blocks= nn.Sequential(*[Block(
            n_head=4, n_embd=n_embd, context_length=context_length
            ) for _ in range(n_layer)]
        )

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        BATCH_SIZE, SEQ_LEN = idx.shape

        tok_emb = self.token_embedding(idx) # (BATCH_SIZE, SEQ_LEN, n_embd)
        pos_emb = self.position_embedding(torch.arange(SEQ_LEN)) # (SEQ_LEN, n_embd)
        x = tok_emb + pos_emb # (BATCH_SIZE, SEQ_LEN, n_embd)
        x = self.blocks(x) # (BATCH_SIZE, SEQ_LEN, n_embd)
        x = self.ln_f(x) # (BATCH_SIZE, SEQ_LEN, n_embd)
        # x = self.ffwd(x) # (BATCH_SIZE, SEQ_LEN, n_embd)
        logits = self.lm_head(x) # (BATCH_SIZE, SEQ_LEN, vocab_size)

        if targets is None:
            loss = None
        else:
            N_EMBD = logits.shape[-1]
            logits = logits.view(BATCH_SIZE*SEQ_LEN, N_EMBD)
            targets = targets.view(BATCH_SIZE*SEQ_LEN)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last context_length tokens
            idx_cond = idx[:, -self.context_length:]

            # get the predictions
            logits, _ = self(idx_cond)

            # get the logits of the last token
            logits = logits[:, -1, :] # (BATCH_SIZE, vocab_size)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1) # (BATCH_SIZE, SEQ_LEN+1)

        return idx